In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import pandas as pd
import numpy as np


# Load dataset
df = pd.read_csv("dataset.csv")
df.head()


,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00


In [ ]:
# Model 1: Linear Pricing
def model1_linear(prev_price, occupancy, capacity, alpha=2.0):
    rate = occupancy / capacity
    new_price = prev_price + alpha * rate
    return max(5, min(new_price, 20))

# Example usage
df["Model1_Price"] = df.apply(lambda row: model1_linear(10, row["Occupancy"], row["Capacity"]), axis=1)
df[["Occupancy", "Capacity", "Model1_Price"]].head()


,Occupancy,Capacity,Model1_Price
0,61,577,10.211438
1,64,577,10.221837
2,80,577,10.277296
3,107,577,10.370884
4,150,577,10.519931


In [ ]:
from geopy.distance import geodesic
import numpy as np

def get_nearby_lots(df, current_row, radius_km=0.5):
    lat1, lon1 = current_row["Latitude"], current_row["Longitude"]

    def is_nearby(row):
        if row.name == current_row.name:
            return False
        try:
            lat2, lon2 = row["Latitude"], row["Longitude"]
            return geodesic((lat1, lon1), (lat2, lon2)).km <= radius_km
        except:
            return False

    return df[df.apply(is_nearby, axis=1)]

In [28]:
def model2_demand_base(row, base_price=10.0):
    α, β, γ, δ, ε = 0.5, 0.3, 0.2, 0.5, 1.0
    occupancy_rate = row.get("occupancy_rate", 0)
    queue_length = row.get("Queue_Length", row.get("queue_length", 0))
    traffic_level = row.get("Traffic_Level", 0)
    is_special_day = row.get("is_special_day", 0)
    vehicle_weight = row.get("vehicle_weight", 0)

    demand = (α * occupancy_rate +
              β * queue_length -
              γ * traffic_level +
              δ * is_special_day +
              ε * vehicle_weight)
    norm_demand = np.tanh(demand)
    price = base_price * (1 + 0.5 * norm_demand)
    return round(max(5, min(price, 20)))

# Apply the model properly
df["Model2_Price"] = df.apply(model2_demand_base, axis=1)
print(df[["Model2_Price"]].head())



   Model2_Price
0            15
1            14
2            15


In [ ]:
# Model 3: Competitive Pricing
def model3_competitive(df, current_row, base_price=10.0):
    nearby_lots = get_nearby_lots(df, current_row)
    model2_price = model2_demand_base(current_row, base_price)
    occupancy = current_row.get("Occupancy", 0)
    capacity = current_row.get("Capacity", 1)  # avoid division by zero

    if not nearby_lots.empty and "Model2_Price" in nearby_lots.columns:
        avg_nearby_price = nearby_lots["Model2_Price"].mean()

        # If the lot is full, price lower than average
        if occupancy >= capacity:
            final_price = min(model2_price, avg_nearby_price - 1)
        # If nearby price is higher, increase your price slightly
        elif avg_nearby_price > model2_price:
            final_price = model2_price + 0.5
        else:
            final_price = model2_price
    else:
        final_price = model2_price

    return round(max(5, min(final_price, 20)), 2)


df["Model3_Price"] = df.apply(lambda row: model3_competitive(df, row), axis=1)

print(df[["Model3_Price"]])

   Model3_Price
0            15
1            14
2            15


In [ ]:
# Save final output
df.to_csv("pricing_output.csv", index=False)
print("Saved pricing output to pricing_output.csv")


Saved pricing output to pricing_output.csv


In [ ]:
import pandas as pd
import numpy as np
import time
from bokeh.io import output_notebook, push_notebook, show
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource

output_notebook()


In [ ]:
# Simulate a simple dataset
n_steps = 20
lots = [f"Lot-{i}" for i in range(1, 6)]

def generate_step_data(step):
    data = []
    for lot in lots:
        occupancy = np.random.randint(0, 50)
        price = 10 + 5 * (occupancy / 50)
        data.append((step, lot, round(price, 2)))
    return pd.DataFrame(data, columns=["time", "lot", "price"])

In [ ]:
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure, show
from bokeh.io import output_notebook, push_notebook

output_notebook()

source = ColumnDataSource(data=dict(time=[], price=[], lot=[]))

p = figure(
    title="Simulated Real-Time Pricing",
    x_axis_label="Time",
    y_axis_label="Price ($)",
    x_range=(0, n_steps),
    y_range=(5, 20),
    width=800,
    height=400  # ✅ use width and height instead of plot_width and plot_height
)

p.scatter(x='time', y='price', source=source, size=8, color='navy', alpha=0.6, marker="circle")

handle = show(p, notebook_handle=True)


In [ ]:
# Streaming simulation loop
for t in range(n_steps):
    df_step = generate_step_data(t)
    for _, row in df_step.iterrows():
        new_data = dict(time=[row["time"]], price=[row["price"]], lot=[row["lot"]])
        source.stream(new_data, rollover=100)
    push_notebook(handle=handle)
    time.sleep(1)